This sample notebook shows how to analyse data fetched from the Data Lake.

It shows the correlation of the average price per minute between TSLA and AAPL in a given day.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import maystreet_data as md
import numpy as np

year, month, day = '2022', '01', '19'

In [ ]:
def avg_price_per_minute():
    """
    Query the Data Lake for the average price per minute.

    Returns a Pandas dataframe with minute_timestamp, avg_aapl_price and avg_tsla_price.
    """

    query = f"""
    SELECT 
        DATE_TRUNC('minute', TO_TIMESTAMP(ExchangeTimestamp / 1000000000)) AS minute_timestamp,
        AVG(CASE WHEN product = 'AAPL' THEN price ELSE NULL END) as avg_aapl_price,
        AVG(CASE WHEN product = 'TSLA' THEN price ELSE NULL END) as avg_tsla_price        
    FROM 
        "prod_lake"."p_mst_data_lake".mt_trade
    WHERE 
        y = '{year}'
        AND m = '{month}'
        AND d = '{day}'
        AND product IN ('AAPL', 'TSLA')
    GROUP BY 1
    ORDER BY 1
    """

    return pd.DataFrame(md.query(md.DataSource.DATA_LAKE, query))

# forward-fill, then backfill in case there was no trade in a given minute
prices = avg_price_per_minute().fillna(method='ffill').fillna(method='bfill')

In [ ]:
# Correlation matrix

np.corrcoef(prices['avg_aapl_price'], prices['avg_tsla_price'])

In [ ]:
# Chart the avg prices

plt.rcParams['figure.figsize'] = [10, 10]

fig, ax = plt.subplots()

fig.patch.set_facecolor((1, 1, 1))

ax.set_xlabel('Avg AAPL price')
ax.set_ylabel('Avg TSLA price')

ax.plot(prices['avg_aapl_price'], prices['avg_tsla_price'], linewidth=0, marker='s')

plt.show()
